Let’s work on the following:
1. Access the CB API (with my login info), which is inside Dropbox. Download the list of companies founded in 2016 and with category ‘Artificial Intelligence’. That should be 500sh companies, we start with a small sample.
2. For each of those companies, download all the NEWS that Crunchbase has. Try to put those news into an easy format: maybe a TXT file per news, and all TXT files of a company into one folder. And remove all non-relevant components (like Ads or other text). This will make it easier if somebody in the team will have to do some reading.
3. Once this looks good, we’ll do some NLP on the text. But let’s get the "raw data" first.
4. Please upload all material on Dropbox.

In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import nltk
from nltk import *
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import sys
import codecs
import string
import random
import re

import requests
import urllib
import bs4
from bs4 import BeautifulSoup
from readability.readability import Document # https://github.com/buriy/python-readability. Tried Goose, Newspaper (python libraries on Github). Bad results.
from http.cookiejar import CookieJar # 

import pycrunchbase
cb = pycrunchbase.CrunchBase('662e263576fe3e4ea5991edfbcfb9883')

/Users/ksun/Downloads/venv3/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
df = pd.DataFrame(columns=["permalink"])

url = 'https://api.crunchbase.com/v3.1/organizations/?user_key=662e263576fe3e4ea5991edfbcfb9883'
# location given here 
location = "philadephia"

# defining a params dict for the parameters to be sent to the API 
PARAMS = {'address':location} 

while True:
    try:
        response = requests.get(url, PARAMS)
        json = response.json()
        for c in json['data']['items']:
            permalink=c['properties']['permalink']
            if(permalink!=None):
                categories_json = requests.get("https://api.crunchbase.com/v3.1/organizations/" 
                                + permalink+"/categories?user_key=662e263576fe3e4ea5991edfbcfb9883", PARAMS).json()
                for cat in categories_json['data']['items']:
                    if(cat["properties"]["category_groups"]!=None):
                        if("Artificial Intelligence" in cat["properties"]["category_groups"]):
                            print(permalink)
                            df = df.append({'permalink' : permalink} , ignore_index=True)
        url = json['data']['paging']['next_page_url'] + "&user_key=662e263576fe3e4ea5991edfbcfb9883"
        print(url)
    except:
        print("error: " + url)
        
df

pulpo
bromin7-inc
sbpa-simulators-technologies
space-hunters
demander-gestão-comercial
james-tip
fdx-finanças-empresariais
greentech-projects
insite-ooh-media-platform-bb11
insite-ooh-media-platform-3d9b
insite-ooh-media-platform
mirage
cere-labs
caravel
esports-charts
esports-charts
hayden-technologies-331a
hayden-technologies-331a
hayden-technologies-331a
streamlytics
streamlytics
jusapi
dosis
interagente
edjo
autaza-tecnologia
anota-ai
anota-ai
valital
shangque
cnvrg-io
cnvrg-io
cnvrg-io
deepwave-digital
deepr-music
simplifai
simplifai
deephealth
deephealth
adx-pro-5152
odm
adchat-ai
adchat-ai
utei
socialbus
bcash
aurit
payhawk
nezaboodka-software
bar-analytics
speedio
keeps-9210
text2data-com
text2data-com
text2data-com
alsodirect-com
cognitive-code
hank-ai-inc
hank-ai-inc
montaaí
neomove
e1ectr0n
c5-face-recognition-cloud-platform
medrec-technologies
smartificial-research-technology-inc
smartificial-research-technology-inc
karv-data-consulting-ltd
karv-data-consulting-ltd
t22
ling

pulpo
bromin7-inc
sbpa-simulators-technologies
space-hunters
demander-gestão-comercial
james-tip
fdx-finanças-empresariais
greentech-projects
insite-ooh-media-platform-bb11
insite-ooh-media-platform-3d9b
insite-ooh-media-platform
mirage
cere-labs
caravel
esports-charts
esports-charts
hayden-technologies-331a
hayden-technologies-331a
hayden-technologies-331a
streamlytics
streamlytics
jusapi
dosis
interagente
edjo
autaza-tecnologia
anota-ai
anota-ai
valital
shangque
cnvrg-io
cnvrg-io
cnvrg-io
deepwave-digital
deepr-music
simplifai
simplifai
deephealth
deephealth
adx-pro-5152
odm
adchat-ai
adchat-ai
utei
socialbus
bcash
aurit
payhawk
nezaboodka-software
bar-analytics
speedio
keeps-9210
text2data-com
text2data-com
text2data-com
alsodirect-com
cognitive-code
hank-ai-inc
hank-ai-inc
montaaí
neomove
e1ectr0n
c5-face-recognition-cloud-platform
medrec-technologies
smartificial-research-technology-inc
smartificial-research-technology-inc
karv-data-consulting-ltd
karv-data-consulting-ltd
t22
linguamatix
linguamatix
morphio
morphio
chemintelligence
chemintelligence
star-mine-technology
corstem
xelpmoc-116d
abstract-304f
abstract-304f
meshcapade
miitis
miitis
miitis
bellweather-solutions
datagen-technologies
listenrobot
listenrobot
onapproach
orca-479a

In [ ]:
# df.to_csv("ai_companies.csv")
# df = df.drop_duplicates(subset ="Permalink", keep = False, inplace = True)
df = pd.read_csv("ai_companies.csv")
df

In [ ]:
# https://api.crunchbase.com/v3.1/organizations?page=43&sort_order=created_at%20DESC&items_per_page=100&user_key=662e263576fe3e4ea5991edfbcfb9883
for i in range(len(df)):
    permalink = df['Permalink'][i]
    newpath = '/Users/ksun/downloads/patent-decision/' + permalink
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    name_of_file = "newsNAME"
    path = newpath + "/"
    completeName = os.path.join(path, name_of_file+".txt")         

    file1 = open(completeName, "w")

    file1.write("stuff")

    file1.close()